# HW 7.5: 自駕車間的溝通 (網路工程) 實作步驟補充

2020.11.19

## 前言

該實作步驟是其中一種方法去完成 `HW7`  
並不是沒有照著做就會做不出來  
所以已經做到一半也沒有必要重來，只需要從中汲取需要的步驟

## Requirement

利用 `socket` 使用 `TCP` 與 `UDP` 協定提供其他聯網裝置與小鴨車溝通，以控制小鴨車馬達與車燈

- 跑 `TCP` 與 `UDP` server 在小鴨車上
- 跑 `app.remote_joystick` 在自己電腦上去控制小鴨車

![](https://imgur.com/sOVVuI1.png)

## Step1: 先在自己電腦，開發 `TCP` server 程式碼

> 如果想直接在小鴨車開發，也是可以的

### 目標

可以將 `TCP` server 跑起來

### 怎麼做

準備一支程式碼為 `TCP` server，另一支程式碼為 `UDP` server
也就是總共有兩個 `Python` 程式碼檔案
如已經打算整合在一起，也是可以的

**這邊先不考慮去引用小鴨車控制函式庫 `car_control.py` 與 `light_control.py`**

程式碼邏輯為:

1. 開啟 `socket`
2. 使用 `bind` 選擇 `host` 與 `port`
3. 使用 `listen` 開始接受 client 連入
4. 用 `accept` 取得與 client 成對的 `socket`
5. 使用 `recv` 取得 client 傳送的資料
6. 先假裝有控制小鴨車，但其實沒有
7. 用 `send` 回覆 client 已處理資料
8. 回到第 5 步

### 注意事項

- 其中使用 `bind` 的時候
  - 將 `host` 改用 `0.0.0.0`，如使用 `localhost` 只能接受同主機的資料
  - 以及將 `port` 改成符合題目描述
- 注意 `accept` 會等到有 client 連上才回傳與 client 成對的 `socket`

## Step2: 使用助教提供的 `remote_joystick.py` 測試 Step1 結果

### 目標

Step1 的 `TCP` server 可以收到 `remote_joystick.py` 傳出去的資料

### 怎麼做

選 **其中一種方法** 將程式碼放到自己電腦上

A. 使用 `git clone -b Lab7 https://github.com/GrassLab/NCTU_CS_ZOO.git Lab7`，適合自己電腦上有 `git` 指令  
B. 在 [`GrassLab/NCTU_CS_ZOO/Lab7`](https://github.com/GrassLab/NCTU_CS_ZOO/tree/Lab7) (https://github.com/GrassLab/NCTU_CS_ZOO/tree/Lab7) 網頁上下載 `ZIP` 壓縮檔，適合自己電腦沒有 `git` 指令

打開終端機，將當前目錄切換到下載下來的程式碼中的 `duckiebot_cs_zoo` 目錄下
輸入指令

```bash
$ python -m app.remote_joystick
Usage: ${程式碼所在目錄}/duckiebot_cs_zoo/app/remote_joystick.py host port
```

這時會看到輸出說用法需要給 `host` 與 `port` 是正常的，因為剛剛的指令的確沒有給  
根據 Step1 所執行程式的機器所在的 `IP` 給予 `host`

如果是跑在自己電腦上，會是

```bash
$ python -m app.remote_joystick ${你自己電腦的 IP} 54760
```

> 如果是跑在小鴨車上，會是
> 
> ```bash
> $ python -m app.remote_joystick duckie-${你的學號}.local 54760
> ```

如果 `remote_joystick` 運作沒有問題，如下:

```bash
connect to ${你自己電腦的 IP}:54760
pressed w, a, s, d and space as joystick when focus on Window
```

就可以按下 `w`, `a`, `s`, `d` 與空白鍵送出指令  
同時應該可以在 Step1 跑起來的 server 去檢查邏輯正確與否

但是遇到問題，就要開始檢查是不是 Step1 實作上有錯，或是 Step2 操作上有錯誤

## Step3: 將 Step1 程式碼放上小鴨車

### 目標

Step1 程式碼可以根據得到的資料完成控制小鴨車馬達

### 怎麼做

連上小鴨車，拉取程式碼

```bash
$ git clone -b Lab7 https://github.com/GrassLab/NCTU_CS_ZOO.git Lab7
```

將 Step1 的程式碼放到 `Lab7/duckiebot_cs_zoo`

先執行 Step1 的程式碼在小鴨車上，並在自己電腦執行 Step2  
確認可以互相交換資料

接下來在程式碼內引用 `car_control`，並使用該函式庫控制馬達

```python
from libs.car_control import CarControl

duckie_car = CarControl(0.0)

# control motor to met velocity and omega
duckie_car.move(velocity=0.5, omega=4.0)
```

速度範圍約為 `-0.8 ~ 0.8`，角速度範圍約為 `-10 ~ 10`

### 注意事項

- 在小鴨車上放置 server 程式碼，記得要在 `duckiebot_cs_zoo` 目錄下，避免引用錯誤 `ImportError`

## Step4: 先在自己電腦，開發 `UDP` server 程式碼

### 目標

可以將 `UDP` server 跑起來

### 怎麼做

大致上與 Step1 很相似，不一樣的地方在於程式碼邏輯

程式碼邏輯為:

1. 開啟 `socket`
2. 使用 `bind` 選擇 `host` 與 `port`
3. 使用 `recvfrom` 取得 client 傳送的資料
4. 先假裝有控制小鴨車，但其實沒有
5. 回到第 3 步

### 注意事項

- 其中使用 `bind` 的時候
  - 將 `host` 改用 `0.0.0.0`，如使用 `localhost` 只能接受同主機的資料
  - 以及將 `port` 改成符合題目描述
- 是使用 `recvfrom` 與 `TCP` 使用的 `recv` 不同

## Step5: 使用助教提供的 `remote_joystick.py` 測試 Step4 結果

### 目標

Step4 的 `UDP` server 可以收到 `remote_joystick.py` 傳出去的資料

### 怎麼做

參考 Step2 去做

差別在於這次要同時把 `TCP` 與 `UDP` server 都跑起來

## Step6: 將 Step4 程式碼放上小鴨車

### 目標

Step4 程式碼可以根據得到的資料完成控制小鴨車車燈

### 怎麼做

應該已經在 Step3 拉取過程式碼

將 Step4 的程式碼放到 `Lab7/duckiebot_cs_zoo`

先執行 Step1 與 Step4 的程式碼在小鴨車上，並在自己電腦執行 Step2  
確認可以互相交換資料

接下來在程式碼內引用 `light_control`，並使用該函式庫控制車燈

```python
from libs.light_control import LightControl

duckie_light = LightControl()

# change front left rgb color to RED
duckie_light.change_rgb(light_pos="FRONT_LEFT", red=1, green=0, blue=0)

# change front left rgb color to RED
duckie_light.change_rgb(light_pos=duckie_light.LightPos.FRONT_LEFT, red=1, green=0, blue=0)

# change front left rgb color to RED
duckie_light.change_rgb(light_pos=0, red=1, green=0, blue=0)
```

`light_pos` 選擇要控制的車燈位置，定義位於 https://github.com/GrassLab/NCTU_CS_ZOO/blob/Lab7/duckiebot_cs_zoo/libs/light_control.py#L26  
`red`, `green`, `blue` 顏色範圍皆為 `0.0 ~ 1.0`  

### 注意事項

- 在小鴨車上放置 server 程式碼，記得要在 `duckiebot_cs_zoo` 目錄下，避免引用錯誤 `ImportError`

# 有問題歡迎到討論區發問